<br>
<br>

![](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

**InstitutoTecnológico y de Estudios Superiores de Occidente**

**Maestría Ciencia de Datos**

**Aprendizaje Profundo**

# Proyecto: clasificación y localización de objetos #

<br>
<br>

* * *

Estudiante: Daniel Nuño <br>
Profesor: Dr. Francisco Cervantes <br>
Fecha entrega: Marzo 26, 2023 <br>

* * *

<br>
<br>

## Libraries

In [ ]:
import os
import random
import cv2 as cv
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.data import AUTOTUNE

from tensorflow.keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

## Pre processing data and functions

Because the folder structure for training is different than validation and test, the pre processing is differently.

### Pre processing training data

Set paths for training and ids-categories

In [ ]:
img_path= "C:/Users/nuno/Desktop/deep-learning-data/proyecto1/tiny-imagenet-200/train"

project_id_path = "C:/Users/nuno/Desktop/deep-learning-data/proyecto1/tiny-imagenet-200/wnids.txt"
all_id_cat_path = "C:/Users/nuno/Desktop/deep-learning-data/proyecto1/tiny-imagenet-200/words.txt"

Read project ids as list. Make sure it has unique values. Make and index of integers from it.

In [ ]:
project_id_list = []
with open(project_id_path) as f:
    for line in f:
        project_id_list.append(line.strip())

Make sure it has unique values.

In [ ]:
len(set(project_id_list)) == len(project_id_list)

Make an index of integers from it.

In [ ]:
project_index_dict = {value: index for index, value in enumerate(project_id_list)}
project_index_dict_by_idx = {index: value for index, value in enumerate(project_id_list)}
len(project_index_dict.values())

Read file of categories as dictionary.

In [ ]:
id_cat_dict = dict()
with open(all_id_cat_path, 'r') as f:
    for line in f:
        resulting_line = line.strip().split('\t')
        id_cat_dict[resulting_line[0]] = resulting_line[1]

Get all list of files while separating bounding box files frome images. Check how many elements has.

In [ ]:
training_files_img = []
training_files_bb = []
for dirpath, dirnames, filenames in os.walk(img_path):
    for filename in filenames:
        path = os.path.join(dirpath, filename)
        if path.endswith('txt'):
            training_files_bb.append(path)
        else:
            training_files_img.append(path)

len(training_files_bb), len(training_files_img)

Process bounding boxes

In [ ]:
bb_dict = dict()
for file in training_files_bb:
    with open(file, 'r') as f:
        for line in f:
            img_name, xmin, ymin, xmax, ymax = line.strip().split('\t')
            bb_dict[img_name] = [xmin, ymin, xmax, ymax]

Check elements of dictionary of bounding boxes

In [ ]:
len(set(bb_dict.keys()))

Create data set list that returns img full path, category, bounding box. Double back slash **'\\'** might needed to be updated depending on the users system.

In tensorflow you can't have a tensor with more than one data type (tf.data.Dataset.from_tensor_slices function). Hence a workaround could be to create a tensor with data type tf.String and, on the occurrence (load_element function below), cast the field to the desired data type.

> It is not possible to have a tf.Tensor with more than one data type. It is possible, however, to serialize arbitrary data structures as strings and store those in tf.Tensors.

In [ ]:
training_list = []
for count, file in enumerate(training_files_img):
    #get category and file name
    _, category_key, _, image_name = training_files_img[count].split('\\')
    #convert category to index from dictionary
    category = project_index_dict[category_key]
    #open image
    img = cv.imread(file)
    #get dimensions
    h, w, _ = img.shape
    #get normalized size bounding box
    original_bb = bb_dict[image_name]
    rs_bb = [float(original_bb[0])/w,
            float(original_bb[1])/h,
            float(original_bb[2])/w,
            float(original_bb[3])/h,
            ]
    # treat it as list
    #example = (file, category, rs_bb)
    #treat it as one string delimited by comas
    example = "".join([file, ',',
                        str(category), ',',
                        str(rs_bb[0]), ',',
                        str(rs_bb[1]), ',',
                        str(rs_bb[2]), ',',
                        str(rs_bb[3])])
    #appended to final list
    training_list.append(example)

len(training_list)

### Pre processing validation data

Path of images and annotations.

In [ ]:
img_path_val = "C:/Users/nuno/Desktop/deep-learning-data/proyecto1/tiny-imagenet-200/val/images"
val_annotations_txt = "C:/Users/nuno/Desktop/deep-learning-data/proyecto1/tiny-imagenet-200/val/val_annotations.txt"

Process annotations that contains category and bounding boxes.

In [ ]:
validation_list = list()
with open(val_annotations_txt, 'r') as f:
    for line in f:
        img_name, category_key, xmin, ymin, xmax, ymax = line.strip().split('\t')
        full_path = img_path_val + '/' + img_name
        category = project_index_dict[category_key]
        img = cv.imread(full_path)
        h, w, _ = img.shape
        rs_xmin = float(xmin)/w
        rs_ymin = float(ymin)/h
        rs_xmax = float(xmax)/w
        rs_ymax = float(ymax)/h
        rs_bb = [rs_xmin, rs_ymin, rs_xmax, rs_ymax]
        #treat it as list
        #example = (full_path, category, rs_bb)
        #treat it as one string delimited by comas
        example = "".join([full_path, ',',
                        str(category), ',',
                        str(rs_bb[0]), ',',
                        str(rs_bb[1]), ',',
                        str(rs_bb[2]), ',',
                        str(rs_bb[3])])
        validation_list.append(example)

Check training and validation have the same format.

In [ ]:
training_list[0]

In [ ]:
validation_list[0]

### Load function and dataset

Because we want to use TensorFlow in batches, it is important to use TensorFlow classes. TensorFlow Dataset allows to load data in small groups, instead of loading all into memory at once.

> Supports writing descriptive and efficient input pipelines. Dataset usage follows a common pattern:

> - Create a source dataset from your input data.
> - Apply dataset transformations to preprocess the data.
>- Iterate over the dataset and process the elements.

> Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

The following function load and treat the image element by element. To be used in the pre-fetched function. Each element is a string delimited by comas:
- full image name, category, ,x min, y min, x max, y max.

- category is an integer from indexing. To use Sparse Categorical Cross Entropy as loss. Expexted to save time in memory as well as computation because it simply uses a single integer for a class, rather than a whole vector. Activation function use softmax and neurons is num_classes. y_pred is [batch_size, num_classes].
- bounding box is a list of 4 float numbers: x_min, y_min, x_max, y_max. Activation function use linear and neurons is 4. y_pred is [batch_size, 4].

The function return a pair on input and targets (category and bounding box). This format is required for *model fit method*:

> Args
> x: Input data. It could be: ...A tf.data dataset. Should return a tuple of either (inputs, targets) or (inputs, targets, sample_weights)...

In [ ]:
def load_element(element):
    #make tensors list delimited by ,
    element = tf.strings.split(element, sep=",")
    #load image
    img = tf.io.read_file(element[0])
    #make sure is 3 channels
    img = tf.image.decode_jpeg(img, channels=3)
    #conver to float [0,1)
    #img = tf.image.convert_image_dtype(img, dtype=tf.float16)
    #resize
    img = tf.image.resize(img, (128, 128))
    #category
    #category = tf.constant(element[1])
    category =tf.strings.to_number(element[1], tf.int32)
    #bounding box
    x_min = tf.strings.to_number(element[2])
    y_min = tf.strings.to_number(element[3])
    x_max = tf.strings.to_number(element[4])
    y_max = tf.strings.to_number(element[5])
    #bb = [y_min, x_min, y_max, x_max]
    bb = [x_min, y_min, x_max, y_max]

    labels = {'class_output': category, 'box_output':bb}

    return (img, labels)

Test the function

In [ ]:
input, labels = load_element(training_list[0])

In [ ]:
input

In [ ]:
labels

Following function **tf.data.Dataset** represents a potentially large set of elements. Iteration in training streaming fashion.

In [ ]:
bath_size = 16
train_dataset = tf.data.Dataset.from_tensor_slices(training_list)
train_dataset = (train_dataset
                 .shuffle(len(training_list))
                 .map(load_element, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .batch(bath_size)
                 .prefetch(AUTOTUNE)
                 )
train_dataset.element_spec

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(validation_list)
val_dataset = (val_dataset
                 .shuffle(len(validation_list))
                 .map(load_element, num_parallel_calls = AUTOTUNE)
                 .cache()
                 .batch(bath_size)
                 .prefetch(AUTOTUNE)
                 )

val_dataset.element_spec

## Define error (loss/metrics) function

### Generalized intersection over union

In [ ]:
def my_GIoU(bb_true, bb_pred):
    #make zero as tensor
    zero = tf.convert_to_tensor(0.0, bb_true.dtype)
    #convert them to tensor clases
    Ax1, Ay1, Ax2, Ay2 = tf.unstack(bb_true, 4, axis=-1)
    Bx1, By1, Bx2, By2 = tf.unstack(bb_pred, 4, axis=-1)

    #for the bounding box predicted make sure Bx2 > Bx1 y By2 > By1
    bx1 = tf.math.minimum(Bx1, Bx2)
    by1 = tf.math.minimum(By1, By2)
    bx2 = tf.math.maximum(Bx1, Bx2)
    by2 = tf.math.maximum(By1, By2)

    #calculate area of true bounding box
    A_area = (Ax2 - Ax1)*(Ay2 - Ay1)
    #calculate area of predicted bounding box
    B_area = (bx2 - bx1)*(by2 - by1)
    
    #calculate intersection over true and pred
    #find the box overlaps both boxes
    #each inter calculates the smallest stride
    x_inter_1 = tf.math.maximum(bx1, Ax1)
    y_inter_1 = tf.math.maximum(by1, Ay1)
    x_inter_2 = tf.math.minimum(bx2, Ax2)
    y_inter_2 = tf.math.minimum(by2, Ay2)
    #get width
    w_inter = tf.maximum(zero, x_inter_2 - x_inter_1)
    #get height
    h_inter = tf.maximum(zero, y_inter_2 - y_inter_1)
    #intersection
    I = w_inter * h_inter
    #area over union
    area_union = (B_area + A_area) - I
    iou = tf.math.divide_no_nan(I, area_union)

    #find the b box C smaller that surrounding/fits both A and B
    Cx1 = tf.math.minimum(bx1, Ax1)
    Cy1 = tf.math.minimum(by1, Ay1)
    Cx2 = tf.math.maximum(bx2, Ax2)
    Cy2 = tf.math.maximum(by2, Ay2)

    #calculate the C area
    C_area = (Cx2 - Cx1) * (Cy2 - Cy1)
    #calculate giou
    giou = iou - tf.math.divide_no_nan(C_area - area_union, C_area)
    #calculate mean of all observations
    m_giou = tf.reduce_mean(giou, axis=0)

    return m_giou

def my_GIoULoss(bb_true, bb_pred):
    return 1.0 - my_GIoU(bb_true, bb_pred)

### Custom classification accuracy error

In [ ]:
def my_sparse_category_accurary(y_true, y_pred):
    #y_true is expected to be integer
    #y_pred is a numpy.ndarray with the normalized probabilities
    acc = np.dot(1, np.not_equal(y_true, np.argmax(y_pred, axis=1)))
    return acc

### Custom bounding box IoU

In [1]:
def my_IoU_np(y_true, y_pred):
    #becausse model predict returns a numpy.ndarray the convert to tensors because le function already works a tensors
    bb_true = tf.convert_to_tensor(y_true, y_true.dtype)
    bb_pred = tf.convert_to_tensor(y_pred, y_pred.dtype)

    #make zero as tensor
    zero = tf.convert_to_tensor(0.0, bb_true.dtype)
    #convert them to tensor clases
    Ax1, Ay1, Ax2, Ay2 = tf.unstack(bb_true, 4, axis=-1)
    Bx1, By1, Bx2, By2 = tf.unstack(bb_pred, 4, axis=-1)

    #for the bounding box predicted make sure Bx2 > Bx1 y By2 > By1
    bx1 = tf.math.minimum(Bx1, Bx2)
    by1 = tf.math.minimum(By1, By2)
    bx2 = tf.math.maximum(Bx1, Bx2)
    by2 = tf.math.maximum(By1, By2)

    #calculate area of true bounding box
    A_area = (Ax2 - Ax1)*(Ay2 - Ay1)
    #calculate area of predicted bounding box
    B_area = (bx2 - bx1)*(by2 - by1)
    
    #calculate intersection over true and pred
    #find the box overlaps both boxes
    #each inter calculates the smallest stride
    x_inter_1 = tf.math.maximum(bx1, Ax1)
    y_inter_1 = tf.math.maximum(by1, Ay1)
    x_inter_2 = tf.math.minimum(bx2, Ax2)
    y_inter_2 = tf.math.minimum(by2, Ay2)
    #get width
    w_inter = tf.maximum(zero, x_inter_2 - x_inter_1)
    #get height
    h_inter = tf.maximum(zero, y_inter_2 - y_inter_1)
    #intersection
    I = w_inter * h_inter
    #area over union
    area_union = (B_area + A_area) - I
    iou = tf.math.divide_no_nan(I, area_union)

    #convert back to numpy
    iou = iou.numpy()
    #check if is over 50% then 0 otherwise 1
    over_50 = np.where(iou>0.5, 0, 1)

    return over_50

### Combine both to return a correct match

In [ ]:
def custom_error(y_true_cat, y_pred_cat, y_true_bb, y_pred_bb):
    d = my_sparse_category_accurary(y_true_cat, y_pred_cat)
    f = my_IoU_np(y_true_bb, y_pred_bb)
    e = np.max(np.column_stack((d,f)), axis=1)
    return e, e.mean()

## Model

Models that are pre trained already did the heavy lifting on getting a tested architecture and fine tuned parameters. Premade architectures with pre-trained weights.

For EfficientNetV2, by default input preprocessing is included as a part of the model (as a Rescaling layer), and thus tf.keras.applications.efficientnet_v2.preprocess_input is actually a pass-through function. In this use case, EfficientNetV2 models expect their inputs to be float tensors of pixels with values in the [0-255] range. At the same time, preprocessing as a part of the model (i.e. Rescaling layer) can be disabled by setting include_preprocessing argument to False. With preprocessing disabled EfficientNetV2 models expect their inputs to be float tensors of pixels with values in the [-1, 1] range.

### Load architecture

We’ll first download the model but exclude the top since we’ll be adding our own custom top and resize the input.

In [ ]:
base_model = tf.keras.applications.EfficientNetV2M(
    include_top=False,
    weights="imagenet",
    input_tensor=Input(shape=(128,128,3)),
    pooling=None,
    include_preprocessing=True)

### Define rest of the multi-output mode for the two tasks

We won’t be training from scratch so we will use transfer learning and split output in two.

- class_output
- box_output

In [ ]:
base_model.trainable=False
base_model_output = base_model.output

no_of_classes = 200

# We could also use Flatten()(x) but GAP is more effective,
# it reduces parameters and controls overfitting.
flattened_output = GlobalAveragePooling2D()(base_model_output)

# Create our Classification Head, final layer contains 
# Ouput units = no. classes
class_prediction = Dense(256, activation="relu")(flattened_output)
class_prediction = Dropout(0.2)(class_prediction )
class_prediction = Dense(no_of_classes, activation='softmax',name="class_output")(class_prediction)

# Create Our Localization Head, final layer contains 4 nodes for x1,y1,x2,y2
# Respectively.
box_output = Dense(256, activation="relu")(flattened_output)
box_output = Dense(128, activation="relu")(box_output)
box_output = Dropout(0.2)(box_output)
box_output = Dense(64, activation="relu")(box_output)
box_output = Dropout(0.2)(box_output)
box_output = Dense(32, activation="relu")(box_output)
box_predictions = Dense(4, activation='sigmoid', name= "box_output")(box_output)

# Now combine the two heads
model = Model(inputs=base_model.input, outputs=[class_prediction, box_predictions])

model.outputs

### Compile and train

In [ ]:
# For classification we will have sparse categorical crossentropy
# For the bouding boxes we will have GIoU
losses = { 
    "class_output": tf.keras.losses.SparseCategoricalCrossentropy(),
    "box_output": my_GIoULoss
    }

# For the class labels we want to know the Accuracy
# And for the bounding boxes we need to know GIoU
metrics = {
    'class_output': tf.keras.metrics.SparseCategoricalAccuracy(), 
    'box_output': my_GIoU
    }

model.compile(optimizer='adam', loss=losses, metrics=metrics)

stop = EarlyStopping(monitor = "val_loss", min_delta = 0.001, patience = 40, 
                    restore_best_weights = True
                     )

model_history = model.fit(x=train_dataset, validation_data=val_dataset, epochs=5, callbacks=stop, verbose=2)

### Evaluate with custom error

Make predictions

In [ ]:
predictions = reconstructed_model.predict(val_dataset)
y_pred_cat, y_pred_bb = predictions


Get true values from dataset

In [ ]:
y_true_cat = np.array([-1])
y_true_bb = np.array([-1, -1, -1, -1])
for element in val_dataset.as_numpy_iterator():
    #y_true_cat.insert(element[1]['class_output'])
    #y_true_bb.insert(element[1]['box_output'])

    y_true_cat = np.append(y_true_cat, element[1]['class_output'])
    y_true_bb = np.append(y_true_bb, element[1]['box_output'])

y_true_cat = y_true_cat[1:]
y_true_bb = y_true_bb[4:]

y_true_bb = y_true_bb.reshape((100,4))

In [ ]:
d, f, e, e_media = custom_error(y_true_cat, y_pred_cat, y_true_bb, y_pred_bb)

Since this is and error, to get an accuracy then use $1-e$ of the model as described in the intructions.

In [ ]:
1 - e_media

### Save model

In [ ]:
model.save('model')

## Test and validation data pipeline

### Reconstruct model

In [ ]:
reconstructed_model = tf.keras.models.load_model('model', custom_objects={'my_GIoULoss':my_GIoULoss, 'my_GIoU':my_GIoU})

### Pipeline for test and new data

1. Define folder path for images
2. Get images names
3. Recursively and for batches:
    - Load image
    - Convert to rgb.
    - Resize
4. make predictions
5. Get category and bounding box from model
    - Get category from dictionary. Dictionary in the form {1:'n02124075'}
    - Convert from normalized to actual size bounding box
6. Save results to a list txt containing the image name, category and bounding box

In [ ]:
def make_predictions_from_folder(path, model, batch_size):
    #2
    img_list = []
    for dirpath, dirnames, filenames in os.walk(img_path):
        for filename in filenames:
            path = os.path.join(dirpath, filename)
            img_list.append(path)
    #3
    def load_test_element(element):
        #load img
        img = tf.io.read_file(element)
        #make sure is 3 channels
        img = tf.image.decode_jpeg(img, channels=3)
        #resize
        img = tf.image.resize(img, (128, 128))
        return img
    
    test_dataset = tf.data.Dataset.from_tensor_slices(img_list)
    test_dataset = (test_dataset
                    .map(load_test_element, num_parallel_calls=AUTOTUNE)
                    .cache()
                    .batch(batch_size)
                    .prefetch(AUTOTUNE)
                    )
    #4
    test_predictions = model.predict(test_dataset)
    cat_pred, bb_pred = test_predictions
    #5.1
    converted_pred_cat = []
    for count, pred in enumerate(cat_pred):
        arg_max = np.argmax(pred)
        converted_pred_cat.append(arg_max)

    return img_list, converted_pred_cat, bb_pred

In [ ]:
def write_file(img_list, cat_pred, bb_pred, dict_cat, file_name):
    #cat_pred is a list or numpy array of integers
    #bb_pred in a list or numpy array of numpy arrays of 4
    converted_pred_cat = []
    for count, pred in enumerate(cat_pred):
        converted_pred_cat.append(dict_cat[pred])
    #5.2
    converted_pred_bb = []
    for count, pred in enumerate(bb_pred):
        #get image path
        file = img_list[count]
        #get real image size
        img = cv.imread(file)
        #get dimensions
        h, w, _ = img.shape
        #correct size bb
        x_min = str(round(pred[0]*w))
        y_min = str(round(pred[1]*h))
        x_max = str(round(pred[2]*w))
        y_max = str(round(pred[3]*h))
        #make a string list
        string = "".join([x_min, '\t',
                        y_min, '\t',
                        x_max, '\t',
                        y_max])
        converted_pred_bb.append(string)
    #6
    output_list = []
    with open(file_name, 'w') as f:
        for count, file in enumerate(img_list):
            _, image_name = file.split('\\')
            output = "".join([image_name, '\t',
                            converted_pred_cat[count], '\t',
                            converted_pred_bb[count]])
            output_list.append(output)
            f.write(output)
            f.write('\n')

    return output_list